In [1]:
from langchain import OpenAI
import json

# Load user data from JSON
with open('userForm.json', 'r') as file:
    user_data = json.load(file)

# Print extracted information
def read_user_data(user_data):
    demographics = user_data['userDetails']['demographicInformation']['data']
    financial_info = user_data['userDetails']['financialInformation']['data']
    retirement_plans = user_data['userDetails']['retirementPlans']['data']
    financial_goals = user_data['userDetails']['financialGoals']['data']
    portfolio_preferences = user_data['portfolioType']['preferences']['data']
    risk_tolerance = user_data['riskTolerance']['tolerance']['data']
    portfolio_structure = user_data['portfolioStructure']['mutualFunds']['data']
    
    return {
        "demographics": demographics,
        "financial_info": financial_info,
        "retirement_plans": retirement_plans,
        "financial_goals": financial_goals,
        "portfolio_preferences": portfolio_preferences,
        "risk_tolerance": risk_tolerance,
        "portfolio_structure": portfolio_structure
    }

# Call the function to extract data
user_info = read_user_data(user_data)
print(user_info)


{'demographics': {'age': {'$numberInt': '25'}, 'gender': 'Male', 'marital': 'Un-married', 'occupation': 'Software Engineer', 'industry': 'IT'}, 'financial_info': {'income': {'$numberInt': '1000000'}, 'investments': ['Real-estate'], 'debt': {'$numberInt': '100000'}, 'insurance': {'$numberInt': '50000'}, 'tax_category': '5 - 15 lakhs'}, 'retirement_plans': {'retirementAge': {'$numberInt': '35'}, 'lifeStyle': ['Buy a new house']}, 'financial_goals': {'timeDuration': {'$numberInt': '10'}, 'investmentGoals': ['Buy a new house']}, 'portfolio_preferences': {'preference': ['Balance long term returns and short term losses']}, 'risk_tolerance': {'tolerance': '56', 'toleranceCategory': 'Moderately High'}, 'portfolio_structure': {'debt': [], 'equity': ['Diversified Equity Funds (Large Cap, Mid Cap, Small Cap, Flexi Cap, Multi Cap)'], 'hybrid': []}}


## Step 2: Creating Reanalysis Agents
## We will create three agents, each handling different aspects of the analysis:
## Agent 1: Income and Investment Analysis

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Initialize OpenAI agent (you can use any LLM here)
llm = OpenAI(temperature=0.9)

# Define a template for the income and investment analysis
income_investment_template = """
Given the following financial data:
- Income: {income}
- Investments: {investments}
- Debt: {debt}

Reassess the risk tolerance level based on these factors.
"""

# Create a prompt template
income_investment_prompt = PromptTemplate(
    template=income_investment_template,
    input_variables=["income", "investments", "debt"]
)

# Create a chain
income_investment_chain = LLMChain(
    llm=llm,
    prompt=income_investment_prompt
)

# Run the chain
income_analysis = income_investment_chain.run({
    "income": user_info['financial_info']['income']['$numberInt'],
    "investments": user_info['financial_info']['investments'],
    "debt": user_info['financial_info']['debt']['$numberInt']
})

print(income_analysis)


## Agent 2: Financial Goals and Preferences

In [ ]:
financial_goals_template = """
Given the following financial goals:
- Goals: {investment_goals}
- Time Duration: {time_duration}

Reanalyze and suggest if the risk tolerance should be adjusted based on these goals.
"""

# Create a prompt template
financial_goals_prompt = PromptTemplate(
    template=financial_goals_template,
    input_variables=["investment_goals", "time_duration"]
)

# Create a chain
financial_goals_chain = LLMChain(
    llm=llm,
    prompt=financial_goals_prompt
)

# Run the chain
goals_analysis = financial_goals_chain.run({
    "investment_goals": user_info['financial_goals']['investmentGoals'],
    "time_duration": user_info['financial_goals']['timeDuration']['$numberInt']
})

print(goals_analysis)


## Agent 3: Risk Tolerance Reanalysis

In [ ]:
risk_tolerance_template = """
Based on the current risk tolerance level of {tolerance} and the target value of {target_value}, 
reassess if any adjustments are needed.
"""

# Create a prompt template
risk_tolerance_prompt = PromptTemplate(
    template=risk_tolerance_template,
    input_variables=["tolerance", "target_value"]
)

# Create a chain
risk_tolerance_chain = LLMChain(
    llm=llm,
    prompt=risk_tolerance_prompt
)

# Run the chain
risk_analysis = risk_tolerance_chain.run({
    "tolerance": user_info['risk_tolerance']['tolerance'],
    "target_value": user_data['riskTolerance']['parameters']['data']['targetValue']['$numberInt']
})

print(risk_analysis)


## Step 3: Compile and Write the New JSON File

In [ ]:
# Collect all analyses and compile results into a dictionary
updated_risk_analysis = {
    "income_investment_analysis": income_analysis,
    "financial_goals_analysis": goals_analysis,
    "risk_tolerance_analysis": risk_analysis
}

# Write the updated data to a new JSON file
with open('updated_userForm.json', 'w') as outfile:
    json.dump(updated_risk_analysis, outfile, indent=4)

print("New risk tolerance values written to updated_userForm.json")
